# Find and explore Powerpoint files from a domain

<p class="alert alert-warning">Work in progress – this notebook isn't finished yet. Check back later for more...<p>

In [153]:
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from tqdm.auto import tqdm
import pandas as pd
import requests_cache
import time
import altair as alt
import os
from urllib.parse import urlparse
from pathlib import Path
import time
# pyMuPDF (aka Fitz) seems to do a better job of converting PDFs to images than pdf2image
import fitz
#from pdf2image import convert_from_path
from sqlite_utils import Database
import sqlite3

# Also need unoconv installed (pip or conda), and LibreOffice installed by whatever means is needed for local system
# We call unoconv from shell below

s = requests.Session()
retries = Retry(total=10, backoff_factor=1, status_forcelist=[ 502, 503, 504 ])
s.mount('https://', HTTPAdapter(max_retries=retries))
s.mount('http://', HTTPAdapter(max_retries=retries))

In [243]:
def get_total_pages(params):
    these_params = params.copy()
    these_params['showNumPages'] = 'true'
    response = s.get('http://web.archive.org/cdx/search/cdx', params=these_params, headers={'User-Agent': ''})
    return int(response.text)

def query_cdx(url, **kwargs):
    results = []
    page = 0
    params = kwargs
    params['url'] = url
    params['output'] = 'json'
    total_pages = get_total_pages(params)
    # print(total_pages)
    with tqdm(total=total_pages-page) as pbar:
        while page < total_pages:
            params['page'] = page
            response = requests.get('http://web.archive.org/cdx/search/cdx', params=params, headers={'User-Agent': ''})
            print(response.url)
            response.raise_for_status()
            data = response.json()
            if data:
                columns = data[0]
                results += data[1:]
            page += 1
            pbar.update(1)
            time.sleep(0.2)
    return columns, results

## Get the CDX data

In [244]:
domain = 'education.gov.au'

In [ ]:
# Domain or prefix search? Domain...
# Collapse on digest? Only removes adjacent captures with the same digest, so probably won't make much difference
# What do we want -- unique combo of urlkey and digest? This will get changes in a file.
# Note the use of regex in the mimetype filter -- should capture all(?) presentations.
columns, results = query_cdx(f'*.{domain}', filter='mimetype:.*(powerpoint|presentation).*')

In [246]:
df = pd.DataFrame(results, columns=columns)

In [247]:
df.shape

(151, 7)

In [249]:
df_unique = df.drop_duplicates(subset=['digest'], keep='first')

In [250]:
df_unique

,urlkey,timestamp,original,mimetype,statuscode,digest,length
0,"au,gov,education,docs)/system/files/doc/other/...",20170317221005,https://docs.education.gov.au/system/files/doc...,application/vnd.openxmlformats-officedocument....,200,4KINVGIFF6MRS333LLSE3H4ZBDVEK74G,219435
3,"au,gov,education,docs)/system/files/doc/other/...",20180423021742,https://docs.education.gov.au/system/files/doc...,application/vnd.openxmlformats-officedocument....,200,EAFDTVCINL4EBHJPCRR5YPZIRKKYKNT6,1009135
5,"au,gov,education,docs)/system/files/doc/other/...",20160329134542,https://docs.education.gov.au/system/files/doc...,application/vnd.openxmlformats-officedocument....,200,TH7PBW7BDPCAQOWECZO6XAW6RE4QM6NS,258215
6,"au,gov,education,docs)/system/files/doc/other/...",20190421165805,https://docs.education.gov.au/system/files/doc...,application/vnd.openxmlformats-officedocument....,200,MA3XCGC6PDQUH34FGOMCDF3JDNDM5NST,1556063
7,"au,gov,education,docs)/system/files/doc/other/...",20180423021849,https://docs.education.gov.au/system/files/doc...,application/vnd.openxmlformats-officedocument....,200,T6JIO3WN4TLI6H4RRWCQVA6JK63GXXH6,2665288
...,...,...,...,...,...,...,...
141,"au,gov,education,heimshelp)/sites/heimshelp/su...",20160312131936,http://heimshelp.education.gov.au/sites/heimsh...,application/vnd.ms-powerpoint.presentation.12,200,M2CXJ4D7QRVPUT6OITVFTWVJWBCF2ZI6,2841389
144,"au,gov,education,heimshelp)/sites/heimshelp/su...",20150329090908,http://heimshelp.education.gov.au/sites/heimsh...,application/vnd.ms-powerpoint.presentation.12,200,WQVVWQQ4ZAWZIGWXCGRIVNK23RMYXCDG,1486559
145,"au,gov,education,heimshelp)/sites/heimshelp/su...",20150329090603,http://heimshelp.education.gov.au/sites/heimsh...,application/vnd.ms-powerpoint.presentation.12,200,4MVZ5MHUIEAQS3D2FYJ4XLRZCVTBLHSK,3395050
146,"au,gov,education,heimshelp)/sites/heimshelp/su...",20160312131731,http://heimshelp.education.gov.au/sites/heimsh...,application/vnd.ms-powerpoint.presentation.12,200,VJW55IJO5PLHIF5HQQJWOUG2WLF2RSQ5,5668594


## Download all the PP files and save some metadata

In [252]:
def get_date_range(df, digest):
    captures = df.loc[df['digest'] == digest]
    return(captures['timestamp'].min(), captures['timestamp'].max())

def check_if_exists(url):
    try:
        response = s.head(url, allow_redirects=True)
    except requests.exceptions.ConnectionError:
        return '404'
    return response.status_code

def save_files(df):
    metadata = []
    unique = df.drop_duplicates(subset=['digest'], keep='first').to_dict('records')
    for row in tqdm(unique):
        url = f'https://web.archive.org/web/{row["timestamp"]}id_/{row["original"]}'
        parsed = urlparse(row["original"])
        suffix = Path(parsed.path).suffix
        # This should give a sortable and unique filename if there are multiple versions of a file
        file_name = f'{slugify(row["urlkey"])}-{row["timestamp"]}{suffix}'
        # print(filename)
        output_dir = Path('powerpoints', slugify(domain))
        output_dir.mkdir(parents=True, exist_ok=True)
        file_path = Path(output_dir, file_name)
        if not file_path.exists():
            response = requests.get(url=url, headers={'User-Agent': ''})
            file_path.write_bytes(response.content)
        details = row.copy()
        first, last = get_date_range(df, row["digest"])
        details['first_capture'] = first
        details['last_capture'] = last
        details['current_status'] = check_if_exists(row["original"])
        details['file_path'] = str(file_path)
        # print(details)
        metadata.append(details)
        time.sleep(5)
    return metadata

In [253]:
metadata = save_files(df)

In [254]:
df_md = pd.DataFrame(metadata)
df_md.to_csv(f'{slugify(domain)}-powerpoints.csv')

## Convert the PP files to PDFs

In [ ]:
# unoconv will sometime hang silently on files it can't convert
# just remove the troublesome files, or change their file extension
# Remove the -v option if you don't want to see the output
input_dir = str(Path('powerpoints', slugify(domain))) + '/*[ppt,pps,pptx]'
output_dir = Path('pdfs', slugify(domain))
output_dir.mkdir(parents=True, exist_ok=True)
pdf_output = str(output_dir)
!unoconv -v -f pdf -o {pdf_output} {input_dir}

## Generate screenshots and save data from PDFs

In [ ]:
import base64

for pdf in Path('pdfs', slugify(domain)).glob('*.pdf'):
    doc = fitz.open(pdf)
    page = doc.loadPage(0) #number of page
    # Define matrix to increase resolution / size - https://pymupdf.readthedocs.io/en/latest/faq/
    pix = page.getPixmap()
    output = str(Path('ppt_images', f'{pdf.stem}.png'))
    pix.writePNG(output)
    b64 = base64.b64encode(pix.getPNGData())
    

In [222]:
import io
from PIL import Image
import PIL

def get_data_from_pdfs(metadata):
    pdf_data = metadata.copy()
    for pdf in tqdm(pdf_data):
        # See if pdf exists
        pdf_name = f'{slugify(pdf["urlkey"])}-{pdf["timestamp"]}.pdf'
        # print(filename)
        output_dir = Path('pdfs', slugify(domain))
        output_dir.mkdir(parents=True, exist_ok=True)
        pdf_path = Path(output_dir, pdf_name)
        if pdf_path.exists():
            doc = fitz.open(pdf_path)
            page_count = doc.pageCount
            page_num = 0
            text = ''
            # Define matrix to increase resolution / size - https://pymupdf.readthedocs.io/en/latest/faq/
            # Better way of fixing size?
            # mat = fitz.Matrix(0.5,0.5)
            while (page_num < page_count):
                page = doc.loadPage(page_num)
                if page_num == 0:
                    pix = page.getPixmap(matrix=mat)
                    output = str(Path('ppt_images', f'{pdf_path.stem}.png'))
                    # pix.writePNG(output)
                    #b64 = base64.b64encode(pix.getPNGData())
                    # pdf['image'] = b64
                    img_data = pix.getPNGData()
                    img = Image.open(io.BytesIO(img_data))
                    ratio = 300 / img.width
                    (width, height) = (round(img.width * ratio), round(img.height * ratio))
                    resized_img = img.resize((width, height), PIL.Image.LANCZOS)
                    #resized_img.save(output)
                    buffer = io.BytesIO()
                    resized_img.save(buffer, format='PNG')
                    pdf['image'] = buffer.getvalue()
                text = text + page.getText()
                pdf['text'] = text
                page_num += 1
        else:
            pdf['text'] = ''
            pdf['image'] = ''
    return pdf_data

In [223]:
pdf_data = get_data_from_pdfs(metadata)

## Save into SQLite, so we can explore the results in Datasette

In [224]:
db = Database("defence-powerpoints.db")

In [225]:
db['files'].insert_all(pdf_data)

<Table files (urlkey, timestamp, original, mimetype, statuscode, digest, length, first_capture, last_capture, current_status, file_path, image, text)>

In [177]:
db.tables

[<Table files (urlkey, timestamp, original, mimetype, statuscode, digest, length, first_capture, last_capture, current_status, file_path, image, text)>]

In [226]:
# There's a slighly modifed version of the render-image plugin in the plugins folder
# The original only showed images < 100kb.
!datasette --plugins-dir plugins defence-powerpoints.db

Serve! files=('defence-powerpoints.db',) (immutables=()) on port 8001
INFO:     Started server process [87382]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8001 (Press CTRL+C to quit)
INFO:     127.0.0.1:50925 - "GET /defence-powerpoints/files HTTP/1.1" 200 OK
INFO:     127.0.0.1:50925 - "GET /-/static/app.css?4434ab HTTP/1.1" 200 OK
INFO:     127.0.0.1:52157 - "GET /defence-powerpoints/files?_next=100 HTTP/1.1" 200 OK
INFO:     127.0.0.1:52157 - "GET /-/static/app.css?4434ab HTTP/1.1" 200 OK
^C
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [87382]
